# Hashprice Calculations

It refers to the expected value of 1 TH/s of hashing power per day **[$/TH/day]**. The metric quantifies how much a miner can expect to earn from a specific quantity of hashrate. You can denominate Hashprice in any currency or asset, but we display the metric in terms of USD or BTC (sats).

Hashprice is a function of four inputs: network difficulty, Bitcoin’s price, block subsidy and transaction fees. Bitcoin’s hashprice will change with every new block added to the blockchain. The Bitcoin Hashprice Index uses a 144 lagging SMA to account for transaction fees.

Hashprice is positively correlated with changes to Bitcoin’s price and transaction fee volume and negatively correlated with changes to Bitcoin’s mining difficulty.

Therefore, to calculate the **Hashprice Index** (HPI), we need:
- Total mining power (hashrate) [EH]      (HR)
- Number of blocks mined per day [-]        (n_blocks)
- Block subsidy (reward) per block [-]      (block_subsidy)
- Daily transaction fees [$]
- BTC Price [$]                             (btc_price)

$$ 
HPI = \frac{BTC_{price}*N_{blocks}*Block_{subsidy}}{HR}
$$

In [10]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import quandl
import yfinance as yf

# Download historical data from Quandl: Bitcoin Difficulty	(DIFF)
df = quandl.get('BCHAIN/MKPRU', api_key='FYzyusVT61Y4w65nFESX').reset_index()

# MIREV: Miners Revenue
# MWNTD: My Wallet Number of Transactions Per Day
# MWNUS: My Wallet Number of Users
# MWTRV: My Wallet Transaction Volume



In [11]:
mining_information_charts = {
    "Currency Statistics": {
        "Market Price": {
            "api_chart_name": "market-price",
            "description": "The average USD market price across major bitcoin exchanges."
        }
    },
    "Network Activity": {
        "Mempool Bytes Per Fee Level": {
            "api_chart_name": "mempool-state-by-fee-level",
            "description": "The current state of the mempool organized by bytes per fee level."
        },
        "Mempool Transaction Count": {
            "api_chart_name": "mempool-count",
            "description": "The total number of unconfirmed transactions in the mempool."
        }
    },
    "Mining Information": {
        "BTC Network Hashrate [TH/s]": {
            "api_chart_name": "hash-rate",
            "description": "The estimated number of terahashes per second the Bitcoin network is performing in the last 24 hours."
        },
        "Hashrate Distribution Over Time": {
            "api_chart_name": "pool-timeseries",
            "description": "An Estimation of hashrate distribution over time amongst the largest mining pools."
        },
        "Network Difficulty": {
            "api_chart_name": "difficulty",
            "description": "A relative measure of how difficult it is to find a new block for the blockchain."
        },
        "Miners Revenue [USD]": {
            "api_chart_name": "miners-revenue",
            "description": "Total value in USD of coinbase block rewards and transaction fees paid to miners."
        },
        "Total Transaction Fees [USD]": {
            "api_chart_name": "transaction-fees-usd",
            "description": "The total USD value of all transaction fees paid to miners. This does not include the coinbase block rewards."
        }
    },
    "Market Signals": {
        "Market Value to Realised Value (MVRV) Ratio": {
            "api_chart_name": "mvrv",
            "description": "The ratio of the market value of Bitcoin to the realized value."
        },
        "Network Value to Transactions (NVT) Ratio": {
            "api_chart_name": "nvt",
            "description": "The ratio of the network value to the USD value of the on-chain transaction volume."
        }
    }
}

chart_name = 'nvt'  # Replace with the desired chart name

def all_charts_available():
    # Define headers for the table with aligned spacing
    print("|----------------------|-----------------------------------------------|---------------------------------------------------------------------------------------------------------------|")
    print("| Category             | Chart                                         | Description                                                                                                   |")
    print("|----------------------|-----------------------------------------------|---------------------------------------------------------------------------------------------------------------|")


    # Iterate over the dictionary to print each chart information
    for category, charts in mining_information_charts.items():
        for chart_name, chart_details in charts.items():
            # Extract chart details
            full_chart_name = chart_name
            api_chart_name = chart_details["api_chart_name"]
            description = chart_details["description"]
            # Print the row in the table format with aligned spacing
            print(f"| {category.ljust(20)} | {full_chart_name.ljust(45)} | {description.ljust(109)} |")

    print("|----------------------|-----------------------------------------------|---------------------------------------------------------------------------------------------------------------|")


# Call the function to print all available charts
all_charts_available()



|----------------------|-----------------------------------------------|---------------------------------------------------------------------------------------------------------------|
| Category             | Chart                                         | Description                                                                                                   |
|----------------------|-----------------------------------------------|---------------------------------------------------------------------------------------------------------------|
| Currency Statistics  | Market Price                                  | The average USD market price across major bitcoin exchanges.                                                  |
| Network Activity     | Mempool Bytes Per Fee Level                   | The current state of the mempool organized by bytes per fee level.                                            |
| Network Activity     | Mempool Transaction Count                     | Th

In [2]:

import json
import requests

# Define the base URL for the API
base_url = 'https://api.blockchain.info/'

def fetch_data_and_save(chart_name, moving_average='2days', timespan='20years', start_date='2009-01-02', format='json', sampling='false'):
    # Construct the API endpoint URL
    charts_url = f'{base_url}charts/{chart_name}?timespan={timespan}&rollingAverage={moving_average}&start={start_date}&format={format}&sampled={sampling}'

    # Make the API call
    response = requests.get(charts_url)

    # Check if the API call was successful
    if response.status_code == 200:
        # Extract the data from the response
        data = response.json()

        # Define the filename
        filename = f'{chart_name.lower()}.json'

        # Save the data to a JSON file
        with open(filename, 'w') as file:
            json.dump(data, file)

        print(f'Data saved to {filename} successfully.')
    else:
        print(f'Failed to fetch data for {chart_name}')

# Call the function for each chart with customizable parameters
def refresh_all_data():        
    for category, charts in mining_information_charts.items():
        for chart_details in charts.items():
            api_chart_name = chart_details["api_chart_name"]
            
            fetch_data_and_save(api_chart_name)

            

In [ ]:
import plotly.graph_objects as go

def plot_chart(api_chart_name, description, log_scale=False):
    filename = f'{api_chart_name.lower()}.json'
    with open(filename, 'r') as file:
        data = json.load(file)

    x = [entry['x'] for entry in data['values']]
    y = [entry['y'] for entry in data['values']]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=api_chart_name))
    
    fig.update_layout(
        title=api_chart_name,
        xaxis_title="Date",
        yaxis_title=description,
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)'  # Darker background color
    )

    # Add a button to toggle between linear and logarithmic scales
    fig.update_layout(
        updatemenus=[
            {
                'buttons': [
                    {
                        'method': 'relayout',
                        'args': [{'yaxis.type': 'linear'}],
                        'label': 'Linear'
                    },
                    {
                        'method': 'relayout',
                        'args': [{'yaxis.type': 'log'}],
                        'label': 'Log'
                    }
                ],
                'direction': 'down',
                'showactive': True,
                'x': 0.1,
                'y': 1.15
            }
        ]
    )

    fig.show()

def plot_all_charts(log_scale=False):
    failed_charts = []
    for category, charts in mining_information_charts.items():
        for chart_name, chart_details in charts.items():
            api_chart_name = chart_details["api_chart_name"]
            description = chart_details["description"]
            try:
                plot_chart(api_chart_name, description, log_scale)
            except Exception as e:
                failed_charts.append(chart_name)
    if failed_charts:
        print(f'Failed to plot the following charts: {", ".join(failed_charts)}')
    else:
        print('All charts plotted successfully.')

# def plot_chart(api_chart_name, description, log_scale=False):
#     filename = f'{api_chart_name.lower()}.json'
#     with open(filename, 'r') as file:
#         data = json.load(file)

#     x = [entry['x'] for entry in data['values']]
#     y = [entry['y'] for entry in data['values']]

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=chart_name))
    
#     fig.update_layout(
#         title=api_chart_name,
#         xaxis_title="Date",
#         yaxis_title=description,
#         template='plotly_dark',
#         yaxis_type='log' if log_scale else 'linear',  # Toggle logarithmic scale
#         plot_bgcolor='rgba(0, 0, 0, 0)'  # Darker background color
#     )
    
#     fig.show()

The Bitcoin network hashrate is an estimate that gauges how much computing power (and thus, miners) are competing for Bitcoin blocks. There’s no way to pull data on the hashrate of all the world’s miners to get a pinpoint figure for the Bitcoin network’s total hashrate. Instead, Bitcoin miners calculate Bitcoin’s estimated total hashrate using Bitcoin’s difficulty and block times.

Hashrate Index charts Bitcoin’s hashrate across three simple-moving-average (SMA) timeframes: 3 days (432 blocks), 7 days (1,008 blocks) and 30 days (4,320 blocks).

The 3 day or 432 blocks timeframe is useful because it is very current. You can easily spot massive disruptions to the Bitcoin mining hashrate from events like China’s Mining Ban, for example. The downside of the 3 day view is that faster or shorter blocks can distort the hashrate estimate, making Bitcoin’s total hashrate appear larger or smaller than it really is.

While less current than the 3 day, the 7 day or 1,008 blocks hashrate metric is less influenced by bitcoin mining luck and block times, and so miners see it as a more accurate estimate. The 7D is the industry standard for hashrate reporting.

Lastly, the 30 day or 4,320 blocks SMA smooths outs most of the noise cause by variance to block times but heavily lags short-term trends.

In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import quandl

# Quandl API key
quandl.ApiConfig.api_key = 'FYzyusVT61Y4w65nFESX'

# Fetch historical Bitcoin price data & hashrate difficulty from Quandl
df = quandl.get('BCHAIN/MKPRU')
df = df.rename(columns={'Value': 'Price'})
df = df.append(quandl.get('BCHAIN/DIFF'))
df = df.rename(columns={'Value': 'Hashrate_Difficulty'})

print(df.head())
# Calculate the block subsidy (daily)
# Block subsidy = 50 BTC (initial subsidy) / (2^floor(years/4))
# The subsidy halves approximately every 4 years
df['Block_Reward'] = 50 / (2 ** (np.floor((df.index.year - 2009) / 4)))
print(df['Block_Reward'])
df['Block_Subsidy'] = df['Block_Reward'] * 144          # 144 blocks per day

# Calculate hashprice index using a 144 lagging SMA for transaction fees
# For simplicity, let's assume transaction fees are constant for now
# You can replace 'Transaction_Fees' with actual transaction fee data if available
df['Hashprice_Index'] = (df['Hashrate_Difficulty']*df['Block_Subsidy']) / 144

# Plot Network Hashrate, Bitcoin Price, and Block Subsidy
fig = go.Figure()

# Plot Hashrate Difficulty
fig.add_trace(go.Scatter(x=df.index, y=np.log(df['Hashrate_Difficulty']), mode='lines', name='Hashrate Difficulty [T]', line=dict(color='blue'), yaxis='y1'))

# # Add secondary y-axis for Bitcoin Price and Block Subsidy
# fig.update_layout(yaxis=dict(title='Network Hashrate (Difficulty)', side='left'),
#                   yaxis2=dict(title='Bitcoin Price', overlaying='y', side='right', anchor='free', position=1))

# Plot Bitcoin Price on secondary y-axis
fig.add_trace(go.Scatter(x=df.index, y=np.log(df['Price']), mode='lines', name='BTC Price [$]' , line=dict(color='green'), yaxis='y2'))

# Plot Block Subsidy
fig.add_trace(go.Scatter(x=df.index, y=df['Block_Reward'], mode='lines', name='Block Subsidy [BTC/day]', line=dict(color='red'), yaxis='y3'))


# Add axis labels
fig.update_layout(title='Bitcoin Network Hashrate, Price, and Block Subsidy over Time',
                  xaxis_title='Date',
                  yaxis1=dict(title='Network Hashrate (Difficulty)', side='left'),
                  yaxis2=dict(title='Bitcoin Price', overlaying='y', side='right', anchor='x'),
                  yaxis3=dict(title='Block Subsidy', overlaying='y', side='right', anchor='free', position=0.85)
                  )

# # Define update function to dynamically update y-axis title
# def update_yaxis_title(visible_traces):
#     if btc_price_trace.visible == True:
#         fig.update_layout(yaxis2=dict(title='Bitcoin Price', overlaying='y', side='right', anchor='free', position=1))
#     elif block_subsidy_trace.visible == True:
#         fig.update_layout(yaxis2=dict(title='Block Subsidy', overlaying='y', side='right', anchor='free', position=2))

# # Define click event handler
# def toggle_visibility(trace, points, state):
#     trace.visible = not trace.visible
#     update_yaxis_title(fig.data)

# # Add click event handler to traces
# for trace in [btc_price_trace, block_subsidy_trace]:
#     trace.on_click(toggle_visibility)

fig.show()


            Price  Hashrate_Difficulty
Date                                  
2009-01-02    0.0                  NaN
2009-01-03    0.0                  NaN
2009-01-04    0.0                  NaN
2009-01-05    0.0                  NaN
2009-01-06    0.0                  NaN
Date
2009-01-02    50.00
2009-01-03    50.00
2009-01-04    50.00
2009-01-05    50.00
2009-01-06    50.00
              ...  
2024-01-03     6.25
2024-01-04     6.25
2024-01-05     6.25
2024-01-06     6.25
2024-01-07     6.25
Name: Block_Reward, Length: 10969, dtype: float64


c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log



In [14]:
print(df['Block_Reward'])

Date
2009-01-02    50.00
2009-01-03    50.00
2009-01-04    50.00
2009-01-05    50.00
2009-01-06    50.00
              ...  
2024-01-03     6.25
2024-01-04     6.25
2024-01-05     6.25
2024-01-06     6.25
2024-01-07     6.25
Name: Block_Reward, Length: 10969, dtype: float64
